!["HCI Banner Logos for ATU Sligo, the HCI Human capital initiative and Higher Education 4.0"](images/HCIBanner.png)

# Modelling the Camera


## Orthographic}
This is fairly simple. 
If your projection is from the origin of the world 3D coordinates then to find the 2D projection you just drop the third coordinate.

This is of limited use as cameras don't really work this way (there are telecentric lenses that do), but it can be a good approximation if your camera position is very far away (theoretically it should be infinity).

So it's a reasonable approximation for telephoto lenses.

## Perspective Projection -  Pinhole Camera Model

The pinhole camera model (and similarly the thin lens model) work on the principle of _similar triangles_.

See the diagram below.

A world point $\mathbf{X} \in \mathbb{R}^3$ is mapped to an image point $\mathbf{x} \in \mathbb{R}^2$.




![](images/pinholeCamera.jpg)

Now we have to decide where the origin is. 

We can decide this arbitrarily as long as we stick to it. 

If the origin is at the camera sensor we have the figure below 


![](images/OriginAtSensor.jpg)

If the origin is at the pinhole we have the figure below. 
 
The formulae are different and as you can see the pinhole version is a little simpler so we will use that one.


![](images/originAtPinhole.jpg)
You will note that the image formed on the sensor is upside down. 

We don't show it, but it shouldn't take too much imagination to see that it will also be be flipped left for right. 

This is not a problem, as we can easily flip these in software or just read off the sensor data upside down and backwards. 

You may have noticed that the front camera on your mobile phone only flips the image vertically but not left for right. 

People are used to seeing themselves in a mirror so if taking a photo of themselves this is how they expect it to behave. 

Most people also prefer their image in a mirror to their actual image, so the readout of the selfie camera can be set to this too.

It is worth noting that this also happens in the human eye and our brain does the flip or we believe it does.
Interestingly, people have done experiments with headsets that flip the image upside down and it only takes a day or two for the brain to adjust to this new norm.


[Clip from BBC documentary, cycling with the world upside down.](https://www.youtube.com/watch?v=-kohUpQwZt8)
 

We can make our lives easier by pretending that the image plane is in front of the pinhole instead of behind. 

The maths still works accept the minus sign is removed. 

See below. This is the idea we will use from now on unless we have to make an exception.


![](images/sensorInFront.jpg)

Now this allows us to link to the idea we used for homogeneous coordinates. 

Our image plane is at $f$ (the focal length). 

The Homogeneous plane can be anywhere, but by convention we used $(Z=1)$. 

If we have a world point at $(X=4, Y=5, Z=3)$ and our focal length is $f=2$ (don't worry about units yet).

Treat the world coordinate as a homogeneous 2D coordinate.

So the 2D coordinates in the plane at ($Z=1$) would be $\frac{4}{3}, \frac{5}{3}$. 

If we project the image on the plane at $Z=1$ out to the focal length $(Z=f=2)$ then we get 2D coordinates  $(\frac{2\times4}{3}, \frac{2\times5}{3})$.

I.e. in the general case we get 2D image coordinates $(\frac{fX}{Z}, \frac{fY}{Z})$.

       

In [1]:
import matplotlib.pyplot as plt

from mpl_toolkits.mplot3d import Axes3D
import numpy as np
import ipywidgets as widgets

# Define a function to update the plot with both elevation and azimuth angles
def update_plot(elev_angle, azim_angle, roll_angle,Xw, Yw, Zw, FL):
    # Create a new matplotlib figure and axis
    fig = plt.figure(figsize=(15, 15))
    ax = fig.add_subplot(111, projection='3d')

    # Set axes labels and limits
    ax.set_xlabel('X axis')
    ax.set_ylabel('Y axis')
    ax.set_zlabel('Z axis')
    ax.set_xlim([-2, 2])
    ax.set_ylim([-2, 2])
    ax.set_zlim([0, 5])

    # Plotting the axes by creating lines for each axis
    axes = [        
        [(-1, 0, 0), (1, 0, 0)],   # x-axis
        [(0, -1, 0), (0, 1, 0)],  # y-axis
        [(0, 0, 0), (0, 0, 5)]  # z-axis
    ]
    colors = ['r', 'g', 'b']  # Colors for each axis
    for ax_line, color in zip(axes, colors):
        ax.plot(*zip(*ax_line), color=color)

    # adding the world coordinate point
    

    world_coord = (Xw, Yw, Zw)

    ax.scatter(*world_coord, color='magenta')
    ax.text(Xw, Yw, Zw, "World coordinate", color='magenta')

    # Drawing a line from the origin to the point
    ax.plot([0, world_coord[0]], [0, world_coord[1]], [0, world_coord[2]], color='magenta')

    # Creating a plane normal to the y-axis centered at (0, 1, 0)
    x = np.linspace(-1, 1, 10)
    y = np.linspace(-1, 1, 10)
    X, Y = np.meshgrid(x, y)
    Z = FL*np.ones_like(X)  # Plane centered at Z=focal length

    # Adding the plane with transparency
    ax.plot_surface(X, Y, Z, color='cyan', alpha=0.2)

    # The intersection point where the magenta line intersects the plane Z = 1

    intersection_point = (FL*Xw/Zw, FL*Yw/Zw, FL*Zw/Zw)
    ax.scatter(*intersection_point, color='magenta')

    # Adjust view
    ax.view_init(elev=elev_angle, azim=azim_angle, roll=roll_angle)
    
    #ax.view_init(elev=30, azim=45, roll=15)
    # Show the plot
    plt.show()

    
elev_slider = widgets.IntSlider(min=0, max=90, step=1, value=10, description='Elevation')
azim_slider = widgets.IntSlider(min=0, max=180, step=1, value=180, description='Azimuth')
roll_slider = widgets.IntSlider(min=-180, max=180, step=1, value=-90, description='Roll')

# Sliders for world coordinates
Xw_slider = widgets.FloatSlider(min=-2, max=2, step=0.1, value=0, description='Xw')
Yw_slider = widgets.FloatSlider(min=-2, max=2, step=0.1, value=0.5, description='Yw')
Zw_slider = widgets.FloatSlider(min=0, max=5, step=0.1, value=3, description='Zw')
FL_slider = widgets.FloatSlider(min=0.1, max=3, step=0.1, value=1, description='Focal Length')
# Interactive widget
widgets.interactive(update_plot, elev_angle=elev_slider, azim_angle=azim_slider, roll_angle=roll_slider, Xw=Xw_slider, Yw=Yw_slider, Zw=Zw_slider, FL=FL_slider)



interactive(children=(IntSlider(value=10, description='Elevation', max=90), IntSlider(value=180, description='…

## Camera Intrinsic and Extrinsic Parameters
So here is what we have to do.
We must go from

World coords (3D) $\to$ Camera coords (3D) $\to$ Image coords (2D) $\to$ Pixel coords (2D Discrete).

So we start with a vector in 3D and we change this to homogeneous Coordinates in 4D. 
We will call this $\textbf{X} = (X, Y, Z, 1)^{\top}$. 

And this is relative to some world coordinate frame, it will be important to know or decide where the origin of the world coordinate frame is.

Next we need to transform this vector to a coordinate relative to the camera origin, which we decided was either the pin hole or the center of the lens. 
This requires the vector to under go rigid body motion based on the **Extrinsic** parameters of the camera. So these include translation and rotation.

So this has moved the vector from the world frame to the camera frame.

In effect what we are doing is moving the 3D origin from the world to the pinhole of the camera and all the vectors must move relative to that. 
The rotation is because the axis of the origin at the camera may be different from the world frame because the camera is rotated in some way.

Next we need to transform the 3D coordinate to a 2D image coordinate. 

This will require a change from 4D Homogeneous to 3D Homogeneous. 

We will simply drop one dimension and divide the first two coordinates by the third and multiply by the focal length. 

Warning: dividing across by the third coordinate is a non-linear operation. 

It's nothing we can't handle, but it makes a mess of our elegant linear algebra. 

For this reason we take it outside the equation and put it on the other side of the equals. It's denoted below as $\lambda$. 

Note that it is not Z as that is the world coordinate that undergoes a lot of changes before it becomes the third coordinate of the final homogeneous image vector.
 



The final thing we need to do is change the 2D coordinates to 2D discrete which means quantizing the space into pixels and usually means shifting the origin to one corner of the sensor. 

The most obvious choice being the top-left of the final image when it has righted itself but this is manufacturer dependent.

The focal length, and the scale factors for the pixels in both x and y are referred to as the **Intrinsic** Camera parameters.
 

Any distortions or differences from the pinhole model are also included under the heading of the camera Intrinsic parameters but may or may not be dealt with directly here and may be modified later in software.


$$\lambda \begin{bmatrix}
                x\\
                y\\
                1
            \end{bmatrix} = \begin{bmatrix}
                s_x    & 0 & O_x  \\
                0     & s_y & O_y  \\
                0     & 0 & 1  
            \end{bmatrix}
                \begin{bmatrix}
                f    & 0 & 0 \\
                0     & f & 0  \\
                0     & 0 & 1  
            \end{bmatrix}
                \begin{bmatrix}
                1    & 0 & 0 & 0 \\
                0     & 1 & 0 & 0 \\
                0     & 0 & 1 & 0 
            \end{bmatrix}
                \begin{bmatrix}
                r_{11}     & r_{12} & r_{13} & t_x \\
                r_{21}     & r_{22} & r_{23} & t_y \\
                r_{31}     & r_{32} & r_{33} & t_z \\
                0 & 0 & 0 & 1
            \end{bmatrix} 
            \begin{bmatrix}
                X\\
                Y\\
                Z\\
                1
\end{bmatrix} $$   

The intrinsic parameters are often put together into one matrix which we will call K.

$$K=\begin{bmatrix}
                fs_x    & 0 & O_x  \\
                0     & fs_y & O_y  \\
                0     & 0 & 1  
            \end{bmatrix}=\begin{bmatrix}
                s_x    & 0 & O_x  \\
                0     & s_y & O_y  \\
                0     & 0 & 1  
            \end{bmatrix}
                \begin{bmatrix}
                f    & 0 & 0 \\
                0     & f & 0  \\
                0     & 0 & 1  
            \end{bmatrix}$$

 




If we put all of these together into one matrix called M we get.

$$ M = \begin{bmatrix}
                m_{11}     & m_{12} & m_{13} & m_{14} \\
                m_{21}     & m_{22} & m_{23} & m_{24} \\
                m_{31}     & m_{32} & m_{33} & m_{34} 
            \end{bmatrix}=
\begin{bmatrix}
                s_x    & 0 & O_x  \\
                0     & s_y & O_y  \\
                0     & 0 & 1  
            \end{bmatrix}
                \begin{bmatrix}
                f    & 0 & 0 \\
                0     & f & 0  \\
                0     & 0 & 1  
            \end{bmatrix}
                \begin{bmatrix}
                1    & 0 & 0 & 0 \\
                0     & 1 & 0 & 0 \\
                0     & 0 & 1 & 0 
            \end{bmatrix}
                \begin{bmatrix}
                r_{11}     & r_{12} & r_{13} & t_x \\
                r_{11}     & r_{12} & r_{13} & t_y \\
                r_{11}     & r_{12} & r_{13} & t_z \\
                0 & 0 & 0 & 1
            \end{bmatrix} $$

$$\lambda \begin{bmatrix}
                x\\
                y\\
                1
            \end{bmatrix} = M\mathbf{X}$$

To calculate the value (x,y) we use


$x=\frac{M_1^{\top}\mathbf{X}}{M_3^{\top}\mathbf{X}}$, $y=\frac{M_2^{\top}\mathbf{X}}{M_3^{\top}\mathbf{X}} $, $z=1$
 

Bear in mind that this transforms only one world point.
And there are an infinite number of world coordinates.
Also keep in mind that any 2D coordinate is the projection along the whole line in 3D ($\lambda x, \lambda y, \lambda $) but only the closest object will appear in the image and it will occlude all points behind it.

The previous detail is useful to aid our understanding of what happens and in particular to help us work back. 

It's actually the matrix M and it's decomposition that is of interest to us to try work our way back from image coordinates to 3D world coordinates.
       

In [11]:
import matplotlib.pyplot as plt
import math
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
import ipywidgets as widgets
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
import matplotlib.patches as patches
from matplotlib.gridspec import GridSpec

camera_coords = np.array(np.zeros([2,2]))
epipole_coords = np.array(np.zeros([2,2]))
lambda1 = 0
lambda2 = 0
x1 = 0
x2 = 0



def update_2d_plots(fig,gs, camera_coords, epipole_coords, ep):
    
    #fig, axs = plt.subplots(1, 2, figsize=(10, 5))
    #axs = [fig.add_subplot(1, 3, 2), fig.add_subplot(1, 3, 3)]
    axs = [fig.add_subplot(gs[0,3]),fig.add_subplot(gs[1,3])]
    # Clear existing plots
    axs[0].cla()
    axs[1].cla()

    # Update and configure the first plot (Camera 1 View)
    axs[0].set_title('Camera 1 View')
    axs[0].set_xlim(-1, 1)
    axs[0].set_ylim(-0.5, 0.5)
    rect1 = patches.Rectangle((-0.6, -0.4), 1.2, 0.8, color=(0, 1, 1, 0.2))  # Cyan color
    axs[0].add_patch(rect1)
    axs[0].scatter(camera_coords[0][0], camera_coords[0][1], color='magenta')
    axs[0].text(camera_coords[0][0], camera_coords[0][1], "x1", color='black')
    if ep:
        axs[0].scatter(epipole_coords[0][0], epipole_coords[0][1], color='black', marker='x')
        axs[0].plot([camera_coords[0][0], epipole_coords[0][0]], [camera_coords[0][1], epipole_coords[0][1]], color='magenta')
    
    

    # Update and configure the second plot (Camera 2 View)
    axs[1].set_title('Camera 2 View')
    axs[1].set_xlim(-1, 1)
    axs[1].set_ylim(-0.5, 0.5)
    rect2 = patches.Rectangle((-0.6, -0.4), 1.2, 0.8, color=(1, 1, 0, 0.2))  # Yellow color
    axs[1].add_patch(rect2)
    axs[1].scatter(camera_coords[1][0], camera_coords[1][1], color='green')
    axs[1].text(camera_coords[1][0], camera_coords[1][1], "x2", color='black')
    if ep:
        axs[1].scatter(epipole_coords[1][0], epipole_coords[1][1], color='black', marker='x')
        axs[1].plot([camera_coords[1][0], epipole_coords[1][0]], [camera_coords[1][1], epipole_coords[1][1]], color='green')
    # Redraw the plots
    plt.draw()


# Define a function to update the plot with both elevation and azimuth angles
def update_plot(elev_angle, azim_angle, roll_angle,Xw, Yw, Zw, FL2, Alpha, Beta, Gamma, tx, ty, tz, ep):
    # Create a new matplotlib figure and axis
    fig = plt.figure(figsize=(20, 6))
    gs = GridSpec(2, 4, figure=fig)
    #fig = plt.figure(figsize=(30, 10))
    #ax = fig.add_subplot(111, projection='3d')
    ax = fig.add_subplot(gs[0:4,:], projection='3d')
    # Set axes labels and limits
    ax.set_xlabel('X axis')
    ax.set_ylabel('Y axis')
    ax.set_zlabel('Z axis')
    ax.set_xlim([-2, 2])
    ax.set_ylim([-2, 2])
    ax.set_zlim([0, 4])
    global E_mat
    global rot


    # Second camera array
    K=np.array([[1,0,0,0],
               [0,1,0,0],
               [0,0,1,0],
               [0,0,0,1]])
    
    T = np.array([[1,0,0,tx],
                   [0,1,0,ty],
                   [0,0,1,tz],
                   [0,0,0,1]])
    
    # Individual Euler angle matrices
    alphaRot = np.array([[1,0,0,0],
       [0,math.cos(math.pi*Alpha/180),-math.sin(math.pi*Alpha/180),0],
       [0,math.sin(math.pi*Alpha/180),math.cos(math.pi*Alpha/180),0],
       [0,0,0,1]])
    betaRot = np.array([[math.cos(math.pi*Beta/180),0,math.sin(math.pi*Beta/180),0],
       [0,1,0,0],
       [-math.sin(math.pi*Beta/180),0,math.cos(math.pi*Beta/180),0],
       [0,0,0,1]])
    gammaRot = np.array([
       [math.cos(math.pi*Gamma/180),-math.sin(math.pi*Gamma/180),0,0],
       [math.sin(math.pi*Gamma/180),math.cos(math.pi*Gamma/180),0,0],
        [0,0,1,0],
       [0,0,0,1]])
    # Full rotation matrix but keep in mind that changing the order will change the rotation.
    rot = alphaRot @ betaRot @ gammaRot
    
    # Camera two focal length only.
    K_FL = ([[FL2,0,0,0],
       [0,FL2,0,0],
       [0,0,FL2,0],
       [0,0,0,1]])
    
    '''Special matrix for the applying the focal length to the z-axis only 
    This is used to move the image sensor with the focal length but not resize the sensor
    '''
    K_plane = ([[1,0,0,0],
               [0,1,0,0],
               [0,0 ,FL2,0],
               [0,0 ,0,1]])
    
    '''K_NF is the camera two matrix but without the focal length
    This is to all the red, green and blue axes for camera two 
    to be the same size as for camera one. So this matrix is to help 
    with the visualisation only'''
    K_NF = K @ T @ rot 
    
    '''K_z is for the visualisation only. It allows the camera two frame to be shown in the correct
    position without re-sizing the frame. Note, as we are only affecting the z-axis, ordering matters here.
    You must do the rotation and translation first and only then extend the z-axis or otherwise you will rotate
    and translate what you did to the z-axis and point it in another direction'''
   
    K_z = K @ T  @ rot @ K_plane 
   
    '''This is the full camera two matrix (relative to camera one). The focal length is in multiples 
    of the first camera focal length. Hence the first camera focal lenght is fixed at 1 and therefore all 
    coordinates are in units of the focal length of camera one'''
    
    K = K  @ T  @  rot @  K_FL  
    
       
    # Plotting the axes for the two cameras
    axes = np.array([[[-.1, 0, 0],[.1, 0, 0]],
            [[0, -.1, 0], [0, .1, 0]],
            [[0, 0, 0], [0, 0, 0.5]]])
               
       
    axes_cam_2 = axes.reshape(6,3)
    axes_cam_2 = np.hstack([axes_cam_2, np.ones((6, 1))])
    axes_cam_2 = K_NF @ axes_cam_2.transpose()
    axes_cam_2 = axes_cam_2.transpose() 
    # Remove the last column
    axes_cam_2 = axes_cam_2[:, :-1]
    axes_cam_2 = axes_cam_2.reshape(3,2,3)
    colors = ['r', 'g', 'b']  # Colors for each axis
    for i in range(0, 3):
        ax.plot([axes_cam_2[i][0][0], axes_cam_2[i][1][0]],  # X coordinates
            [axes_cam_2[i][0][1], axes_cam_2[i][1][1]],  # Y coordinates
            [axes_cam_2[i][0][2], axes_cam_2[i][1][2]],  # Z coordinates
            color=colors[i]) 
        
        ax.plot([axes[i][0][0], axes[i][1][0]],  # X coordinates
            [axes[i][0][1], axes[i][1][1]],  # Y coordinates
            [axes[i][0][2], axes[i][1][2]],  # Z coordinates
            color=colors[i])   
     
    
    
    # adding the world coordinate point
    world_coord = np.array([Xw, Yw, Zw])
    ax.scatter(*world_coord, color='black')
    ax.text(Xw, Yw, Zw, "X", color='black')

    # Drawing a line from the origin to the  World coordinate point
    ax.plot([0, world_coord[0]], [0, world_coord[1]], [0, world_coord[2]], color='magenta')
    
    # Creating a plane normal to the y-axis centered at (0, 1, 0)
    x = np.linspace(-.6, .6, 10)
    y = np.linspace(-.4, .4, 10)
    X, Y = np.meshgrid(x, y)
    Z = np.ones_like(X)  # Plane centered at Z=focal length
    image_plane1 = np.array([X,Y,Z, np.ones_like(X)])
    
    
    
    camera_2_center = K_NF @ np.array([0,0,0,1])
    # Drawing a line from the camera 2 center to the point
    ax.plot([camera_2_center[0], world_coord[0]], [camera_2_center[1], world_coord[1]], [camera_2_center[2], world_coord[2]], color='green')
    
    # Adding the plane with transparency
    ax.plot_surface(image_plane1[0], image_plane1[1], image_plane1[2], color='cyan', alpha=0.2)
    
    # This reshapes image_plane1 for matrix multiplication by our camera 2 matrix
    image_plane2 = K_z @ image_plane1.reshape(4,-1) 
    
    # Reshaping back to original shape
    image_plane2 = image_plane2.reshape(image_plane1.shape) 
    ax.plot_surface(image_plane2[0], image_plane2[1], image_plane2[2], color='yellow', alpha=0.2)
    
    # The intersection point where the magenta line intersects the image_plane1 Z = 1
    intersection_point = (Xw/Zw, Yw/Zw, Zw/Zw)
    cam_1_coord = np.array(intersection_point[:2])
    
    
    ax.scatter(*intersection_point, color='magenta')
    world_hom = np.array([Xw,Yw,Zw,1])
    try:
        K_inv = np.linalg.inv(K)
    except np.linalg.LinAlgError:
        print("The matrix is not invertible.")
        
    temp_world = K_inv @ world_hom
    
    intersection_point_imageP2 = np.array([FL2*temp_world[0]/temp_world[2], 
                                  FL2*temp_world[1]/temp_world[2], 
                                  FL2*temp_world[2]/temp_world[2],1])
    
    x2 = intersection_point_imageP2[:3]
    cam_2_coord = intersection_point_imageP2[:2]
    
    intersection_point_imageP2 = K_NF @ intersection_point_imageP2
    pt = (intersection_point_imageP2[0],intersection_point_imageP2[1],intersection_point_imageP2[2])
    ax.scatter(*pt, color='green')
    
    # draw line between camera centers
    ax.plot([0, camera_2_center[0]], [0, camera_2_center[1]], [0, camera_2_center[2]], color='cyan')
    
    points = np.array([[0, 0, 0],  # Origin - camera 1 center
                       [world_coord[0], world_coord[1], world_coord[2]],  # World coordinate
                       [camera_2_center[0], camera_2_center[1], camera_2_center[2]]])  # Camera 2 center

    # Shade in the Epipolar plane
    epipoloar_plane = Poly3DCollection([points])
    epipoloar_plane.set_color('grey')
    epipoloar_plane.set_alpha(0.2)  # Adjust transparency here
    ax.add_collection3d(epipoloar_plane)
    
    # Show the epipole for camera 1
    cam_1_epipole = (camera_2_center[0]/camera_2_center[2], 
                     camera_2_center[1]/camera_2_center[2], 
                     camera_2_center[2]/camera_2_center[2])
    ax.scatter(*cam_1_epipole, color='black', marker='x')
    epipole_coords[0] = np.array([cam_1_epipole[0], cam_1_epipole[1]])
    
    # Show the epipole for camera 2
    cam_2_view_origin = K_inv @ np.array([0,0,0,1])
    cam_2_epipole = (cam_2_view_origin[0]/cam_2_view_origin[2], 
                     cam_2_view_origin[1]/cam_2_view_origin[2],
                     cam_2_view_origin[2]/cam_2_view_origin[2], 1)
    epipole_coords[1] = np.array([cam_2_epipole[0], cam_2_epipole[1]])
    cam_2_epipole = K @ cam_2_epipole
    ax.scatter(*cam_2_epipole[:3], color='black', marker='x')
    
    # Adjust view
    ax.view_init(elev=elev_angle, azim=azim_angle, roll=roll_angle)
    
    #show view in camera 1    
    cam_1_coord = np.array([world_coord[0]/world_coord[2], world_coord[1]/world_coord[2]])
    
    
    camera_coords[0] = cam_1_coord
    camera_coords[1] = cam_2_coord
    
    
    
    update_2d_plots(fig,gs, camera_coords, epipole_coords, ep)
    
    lambda1 = world_coord[2]#math.sqrt((world_coord[0]**2)+(world_coord[1]**2)+(world_coord[2]**2))
    lambda2 = FL2*temp_world[2]# math.sqrt((world_coord[0]-camera_2_center[0])**2+(world_coord[1]-camera_2_center[1])**2+(world_coord[2]-camera_2_center[2])**2)
    
    
    print(f'lambda1:{lambda1}')
    print(f'lambda2:{lambda2}')
    
    
    x1 = np.append(cam_1_coord, 1)
    print(f'x1:{x1}')
    #np.append(cam_2_coord, 1)
    print(f'x2:{x2}')
    Tx = np.array([[0, -tz, ty],
                   [tz, 0, -tx],
                   [-ty, tx, 0]])
    
    R = np.array(rot[:3,:3])
    print(f'x1TxRx2:{ x1 @ Tx @ R @ x2}')
    print(f'lambda1*x1:{lambda1*x1}')
    R_inv = np.linalg.inv(R)
    print(f'R(lambda1*x1)+T:{R_inv @ (lambda1*x1 -np.array([tx, ty,tz])) }' )
    print(f'lambda2*x2:{lambda2*x2}')
    
    E_mat= Tx @ R
    
    
    print(f'Determinant of E=TxR: {np.linalg.det(E_mat)}')
    # Show the plot
    plt.show()
    



elev_slider = widgets.IntSlider(min=-180, max=180, step=1, value=0, description='Elevation')
azim_slider = widgets.IntSlider(min=-180, max=180, step=1, value=90, description='Azimuth')
roll_slider = widgets.IntSlider(min=-180, max=180, step=1, value=-90, description='Roll')

# Sliders for world coordinates
Xw_slider = widgets.FloatSlider(min=-2, max=2, step=0.1, value=0, description='Xw')
Yw_slider = widgets.FloatSlider(min=-2, max=2, step=0.1, value=0.5, description='Yw')
Zw_slider = widgets.FloatSlider(min=0, max=5, step=0.1, value=3, description='Zw')

FL_slider2 = widgets.FloatSlider(min=0.1, max=3, step=0.1, value=1.0, description='Cam 2 Focal')

alpha_slider = widgets.IntSlider(min=-180, max=180, step=1, value=0, description='Cam2 Alpha')
beta_slider = widgets.IntSlider(min=-180, max=180, step=1, value=-150, description='Cam2 Beta')
gamma_slider = widgets.IntSlider(min=-180, max=180, step=1, value=-70, description='Cam2 Gamma')


tx_slider = widgets.FloatSlider(min=-2.0, max=2.0, step=0.1, value=1, description='Tx')
ty_slider = widgets.FloatSlider(min=-2.0, max=2.0, step=0.1, value=1, description='Ty')
tz_slider = widgets.FloatSlider(min=0.0, max=5.0, step=0.1, value=4, description='Tz')

# Group sliders into two columns
left_box = widgets.VBox([elev_slider, azim_slider, roll_slider, Xw_slider, Yw_slider, Zw_slider ])
right_box = widgets.VBox([ FL_slider2, alpha_slider, beta_slider, gamma_slider, tx_slider, ty_slider, tz_slider])

epipoles_checkbox = widgets.Checkbox(value=False, description='Show Epipoles in 2D',disabled=False)

# Combine the two columns into a single horizontal layout
ui = widgets.HBox([left_box,  right_box])


# Interactive widget
out = widgets.interactive_output(update_plot, {'elev_angle': elev_slider, 'azim_angle': azim_slider, 
                                               'roll_angle': roll_slider, 'Xw': Xw_slider, 
                                               'Yw': Yw_slider, 'Zw': Zw_slider, 'FL2': FL_slider2, 
                                               'Alpha': alpha_slider, 'Beta': beta_slider, 'Gamma': gamma_slider,
                                              'tx': tx_slider, 'ty': ty_slider, 'tz': tz_slider, 'ep': epipoles_checkbox})

sliders_box = widgets.VBox([elev_slider, azim_slider, roll_slider, 
                            Xw_slider, Yw_slider, Zw_slider,
                            FL_slider2, alpha_slider, beta_slider, 
                            gamma_slider, tx_slider, ty_slider, tz_slider, epipoles_checkbox])
ui = widgets.HBox([sliders_box, out])


# Display the UI and the output widget
display(ui)

## Distortions

The pin hole camera model is certainly a very good approximation but as we know most cameras use lenses to gather more light. 
How much do these differ from the pinhole model?
Well firstly, with a lens, not everything is in focus.
Secondly there are all sorts of distortions which can affect the image.

I'll mention radial distortions here but I recommend  to anyone that is interested in lens distortions to look up [Marc Levoy's Stanford slides](http://graphics.stanford.edu/courses/cs478/lectures/01112012_optics.pdf)
on the limitations of lenses or to read [Eugene Hecht's book on Optics](https://www.amazon.co.uk/Optics-Global-Eugene-Hecht/dp/1292096934/ref=sr_1_1?ie=UTF8&qid=1550318092&sr=8-1&keywords=hecht+optics)

       



## Radial Distortion
I mention this here as this has a large effect on image coordinates. 

Radial distortion is distortion that has an effect that is related to the distance from the center of the lens. 

This is most likely in lenses with a wide field of view i.e. a very short focal length. 

Fish-eye lenses are an extreme version of this.

The basic problem is that the rules of projective geometry are broken i.e. straightness of lines are not preserved. 


## Radial Distortion
The examples below show a (synthetic) radial distortion in order to drive home the point. 

What we notice is that the straight lines in the real world are curved in the image and this curving becomes more pronounced the further we get from the center.


![](images/DCURadialDistortion.png)
 

!["HigherEd 4.0 is funded by the Human Capital Initiative Pillar 3. HCI Pillar 3 supports projects to enhance the innovation and agility in response to future skills needs"](images/HCIFunding.png)